In [ ]:
!pip install -q  fastai==1.0.61
!pip install efficientnet_pytorch torchtoolbox

In [ ]:
import fastai
fastai.__version__

In [ ]:
from pathlib import Path
import random
from fastai.callbacks import SaveModelCallback
from fastai.basic_data import DatasetType
from fastai.callbacks import SaveModelCallback
from fastai.metrics import accuracy
from fastai.train import ClassificationInterpretation
from fastai.vision.data import ImageList
from fastai.vision.learner import cnn_learner
from fastai.vision.models import resnet50
from fastai.vision.transform import ResizeMethod, get_transforms
from fastai.vision import imagenet_stats

import pandas as pd

In [ ]:
path = Path('../input/ensi-hackathon')
il = ImageList.from_csv(path, 'train_ids_labels.csv', folder='all_data/all_data', suffix='.JPG')
sd = il.split_by_rand_pct(0.15)
ll = sd.label_from_df()
ll = ll.transform(get_transforms(), size=300)
data = ll.databunch(bs=64).normalize(imagenet_stats)

In [ ]:
test = (ImageList.from_csv(path, 'test_ids_only.csv', folder='all_data/all_data', suffix='.JPG'))
data.add_test(test)

In [ ]:
data

In [ ]:
data.show_batch(3)

In [ ]:
import torch.nn.functional as F
learn = cnn_learner(data, resnet50, metrics=[accuracy,F.cross_entropy])
learn = learn.mixup()

In [ ]:
#!chmod 775 '../input/ensi-hackathon'

In [ ]:
#learn.loss_func
#learn.opt_func

In [ ]:
lr = 1e-2
learn.fit_one_cycle(7, lr)

In [ ]:
preds, y = learn.get_preds(DatasetType.Test)

In [ ]:
fnames = [o.split('/')[-1].split('.')[0] for o in learn.data.test_ds.items]

In [ ]:
subm = pd.DataFrame({'Image_ID': fnames, 'Label': preds[:, 1]})

In [ ]:
s=subm.to_csv('github_model.csv', index=None)
s=pd.read_csv('./github_model.csv')
s.head()